# 2023-2 언어데이터과학 24강 (2023-11-29) 실습 (1) `gensim` 패키지와 Word2Vec 모델 훈련

## 오늘의 목표

1. 모두의 말뭉치 '온라인 대화 말뭉치'의 발화 자료로 Word2Vec Skip-gram 모델을 훈련시킬 수 있다.
2. 단어 사이의 의미 유사성을 벡터들의 코사인 유사도로 계량화하여 설명할 수 있다.
3. t-SNE 기법을 사용하여 단어 벡터들을 2차원 평면에 시각화할 수 있다.
4. Word2Vec 모델을 사용하여 연령대에 따른 단어의 분포 변화를 추적할 수 있다.

## 0. 데이터 가공

12–15강에서 만든 모두의 말뭉치 [온라인 대화 말뭉치] 파일을 읽고 `gensim` 모듈에서 사용 가능한 코퍼스로 가공하자.

In [3]:
import pandas as pd
from gensim.models import Word2Vec
from tqdm import tqdm
from gensim.models import FastText

### 데이터 파일 읽기

In [2]:
DATA_PATH = '../../data/NIKL_OM_form_age_sex.csv.tar.gz'

In [4]:
utterances = pd.read_csv(DATA_PATH, compression='gzip', on_bad_lines='skip')
utterances

,data/NIKL_OM_form_age_sex.csv,form,speaker_id,age,sex
0,MDRW2100000001.1.1,안녕하세요,MDRW2100000001_1,20대,여성
1,MDRW2100000001.1.4,이거 해봐요><,MDRW2100000001_1,20대,여성
2,MDRW2100000001.1.7,오 ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ,MDRW2100000001_1,20대,여성
3,MDRW2100000001.1.8,안챙겨도 잘커요,MDRW2100000001_1,20대,여성
4,MDRW2100000001.1.9,너무 맞는데요ㅜㅜ?,MDRW2100000001_1,20대,여성
...,...,...,...,...,...
2977836,MMRW2100000241.1.2775,한 번도 안 써봄...?,MMRW2100000241_2,20대,여성
2977837,MMRW2100000241.1.2776,그거 개꿀인디,MMRW2100000241_2,20대,여성
2977838,MMRW2100000241.1.2780,ㅋㅋㅋㅋㅋㅋ잠수복 개귀여웤ㅋㅋㅋㅋ,MMRW2100000241_2,20대,여성
2977839,MMRW2100000241.1.2786,ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ가즈아!,MMRW2100000241_2,20대,여성


In [5]:
utterances = pd.read_csv(DATA_PATH, compression='gzip', on_bad_lines='skip')
utterances.dropna(inplace=True)
utterances.rename(columns={utterances.columns[0]: 'id'}, inplace=True)
utterances.set_index('id', inplace=True)
utterances

,form,speaker_id,age,sex
id,,,,
MDRW2100000001.1.1,안녕하세요,MDRW2100000001_1,20대,여성
MDRW2100000001.1.4,이거 해봐요><,MDRW2100000001_1,20대,여성
MDRW2100000001.1.7,오 ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ,MDRW2100000001_1,20대,여성
MDRW2100000001.1.8,안챙겨도 잘커요,MDRW2100000001_1,20대,여성
MDRW2100000001.1.9,너무 맞는데요ㅜㅜ?,MDRW2100000001_1,20대,여성
...,...,...,...,...
MMRW2100000241.1.2774,그 낚시대회 전용 투망 있을걸???,MMRW2100000241_2,20대,여성
MMRW2100000241.1.2775,한 번도 안 써봄...?,MMRW2100000241_2,20대,여성
MMRW2100000241.1.2776,그거 개꿀인디,MMRW2100000241_2,20대,여성


### 데이터 형변환

일반적으로 Python의 여러 라이브러리에서 코퍼스를 다룰 때는 한 문장을 단어들의 리스트로 표현하고, 코퍼스 전체를 문장들의 리스트로 표현한다.

지금 가지고 있는 데이터프레임에서는 문장에 해당하는 발화가 `str` 자료형이므로, `str.split()` 메소드를 사용하여 단어들의 리스트로 만들어 주자.

In [6]:
corpus = utterances['form'].apply(str.split) # edit this line
print(corpus[:5])

id
MDRW2100000001.1.1            [안녕하세요]
MDRW2100000001.1.4        [이거, 해봐요><]
MDRW2100000001.1.7    [오, ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ]
MDRW2100000001.1.8        [안챙겨도, 잘커요]
MDRW2100000001.1.9      [너무, 맞는데요ㅜㅜ?]
Name: form, dtype: object


## 1. FastText 모델 훈련

### 모델 초기화

In [7]:
model_ft = FastText(
    sg=1, #skit-gram
    min_count=5,#코퍼스에서 5번 미만으로 나온 단어는 생략 +
    vector_size=100, #vector dimension, 벡터 사이즈가 많을수록 단어 하나를 많은 수의 실수값으로 나타내는 것이기에 더 정교해짐, 하지만 시간과 저장 공간이 늘어나 
    window=2, # 앞의 2 단어 까지만 보겠음!
    negative=5,
)

### Vocabulary 구축

In [8]:
model_ft.build_vocab(corpus_iterable=tqdm(corpus))

  4%|▍         | 120424/2977840 [00:00<00:04, 599762.27it/s]

100%|██████████| 2977840/2977840 [00:06<00:00, 459890.30it/s]


### 모델 훈련시키기

In [9]:
model_ft.train(
    corpus_iterable=corpus,
    total_examples=model_ft.corpus_count, # the number of sentences
    epochs=5 #코퍼스를 다섯 번 읽어라
)

(32844172, 46433495)

## 2. Word2Vec 모델 파일 불러오기

In [10]:
MODEL_PATH = '../../models/word2vec-modu-online'

model_wv = Word2Vec.load(MODEL_PATH)

## 3. FastText 모델에서 유사도 계산

In [11]:
# Word2Vec
print(model_wv.wv.most_similar(positive=['할머니', '아빠'], negative=['엄마'])) # ADD THIS LINE

[('할아버지', 0.8393165469169617), ('초등학교', 0.8178285956382751), ('name10', 0.8162828087806702), ('이모가', 0.8138353228569031), ('너희', 0.8085528016090393), ('name12', 0.8033748269081116), ('name11', 0.800075113773346), ('니네', 0.7999460697174072), ('name9', 0.7989993691444397), ('형부', 0.7969768047332764)]


In [12]:
#FastText
print(model_ft.wv.most_similar(positive=['할머니', '아빠'], negative=['엄마'])) # ADD THIS LINE

[('할머니나', 0.8682805895805359), ('할머니만', 0.8610612154006958), ('할머니와', 0.8580013513565063), ('할머니댁', 0.8543075323104858), ('할머니의', 0.8542176485061646), ('할머니들', 0.8522158265113831), ('할머니를', 0.8480395078659058), ('할머니네', 0.8450437784194946), ('할머니랑', 0.8448000550270081), ('할머니댁에', 0.8434090614318848)]


## model_wv와 model_ft비교

In [21]:
model_wv.wv.most_similar(['완전'])

[('짱', 0.7086948156356812),
 ('되게', 0.6537210941314697),
 ('대박', 0.6253469586372375),
 ('넘나', 0.6242092251777649),
 ('무지', 0.6102795600891113),
 ('디게', 0.6088290810585022),
 ('겁나', 0.6073566675186157),
 ('왕', 0.6060531139373779),
 ('아주', 0.5990931391716003),
 ('진짜', 0.593711793422699)]

In [22]:
model_ft.wv.most_similar(['완전'])

[('나완전', 0.9183516502380371),
 ('완전완전', 0.8679131865501404),
 ('완전ㅠ', 0.8432345390319824),
 ('완전!', 0.7912197113037109),
 ('완전체', 0.7200117707252502),
 ('완전요', 0.7129219174385071),
 ('완전좋아', 0.6801377534866333),
 ('짱', 0.6633487939834595),
 ('완전체로', 0.6570025682449341),
 ('되게', 0.6539560556411743)]

In [23]:
model_wv.wv.most_similar(['아주'])

[('매우', 0.7686690092086792),
 ('굉장히', 0.667973518371582),
 ('무척', 0.6672804951667786),
 ('무지', 0.6570212244987488),
 ('상당히', 0.6478555798530579),
 ('되게', 0.6371939182281494),
 ('정말', 0.6210654973983765),
 ('넘나', 0.6183302402496338),
 ('디게', 0.6104277968406677),
 ('완전', 0.5990931987762451)]

In [24]:
model_ft.wv.most_similar(['아주'])

[('아주아주', 0.7850839495658875),
 ('아주~', 0.7354899048805237),
 ('매우', 0.7182841897010803),
 ('아주대', 0.7006793022155762),
 ('굉장히', 0.6741524934768677),
 ('아아주', 0.6675925254821777),
 ('아주조아', 0.6643998622894287),
 ('무척', 0.6612856388092041),
 ('아주좋아', 0.6600858569145203),
 ('매우매우', 0.6521584987640381)]

In [25]:
model_wv.wv.most_similar(['^^'])

[('~', 0.8403391242027283),
 ('ㅎㅎㅎㅎ', 0.8390881419181824),
 (':)', 0.8387506604194641),
 ('~~', 0.8380871415138245),
 ('ㅎㅎㅎ', 0.8291387557983398),
 ('히히', 0.8150464296340942),
 ('!', 0.7977784872055054),
 ('!!', 0.7945547699928284),
 ('ㅎ.ㅎ', 0.787035346031189),
 ('헤헤', 0.7769703269004822)]

In [27]:
model_ft.wv.most_similar(['^^'])

[('^^^', 0.8581323027610779),
 ('^^7', 0.8380123376846313),
 ('..^^', 0.8265505433082581),
 ('ㅎㅎ^^', 0.8249148726463318),
 ('^^^^', 0.8245590329170227),
 (':)', 0.8199918270111084),
 ('~~', 0.8174892067909241),
 ('~~~^^', 0.816228985786438),
 ('~~^^', 0.81227707862854),
 ('^^!', 0.8088483810424805)]

In [26]:
model_wv.wv.most_similar(['ㅋㅋ'])

[('ㅋㅋㅋ', 0.9428948163986206),
 ('ㅋㅋㅋㅋ', 0.8743140697479248),
 ('ㅋㅋㅋㅋㅋ', 0.8505040407180786),
 ('ㅋㅋㅋㅋㅋㅋ', 0.8215165138244629),
 ('ㅋㅋㅋㅋㅋㅋㅋ', 0.7947203516960144),
 ('ㅎㅎㅎ', 0.7686710357666016),
 ('ㅋㅋㅋㅋㅋㅋㅋㅋ', 0.7684953808784485),
 ('ㅎㅎㅎㅎ', 0.767102062702179),
 ('ㅎㅎ', 0.7521860003471375),
 ('ㅋㅋㅋㅋㅋㅋㅋㅋㅋ', 0.7333099246025085)]

In [28]:
model_ft.wv.most_similar(['ㅋㅋ'])

[('ㅋㅋㅋ', 0.8951324820518494),
 ('ㅋㅋㅋㄱㄱㅋㅋ', 0.8187497854232788),
 ('ㅋㅋㅋ엌ㅋㅋ', 0.813736081123352),
 ('ㅋㅋㅋㅌㅋㅋ', 0.811890721321106),
 ('ㅋㅋㅋㄲㅋㅋ', 0.810799777507782),
 ('ㅋㅋㅋㄱ', 0.8019217252731323),
 ('ㅋㅋㅋㄱㅋㅋ', 0.8017099499702454),
 ('ㅋㅋㄱ', 0.8008558750152588),
 ('ㅋㅋㅋ!', 0.8000901341438293),
 ('ㅋㅋㅌㅋㅋㅋ', 0.7987284064292908)]

In [13]:
model_wv.wv.most_similar(['너뮤'])

[('상상만해도', 0.9265857934951782),
 ('생각만해도', 0.9127776026725769),
 ('죽겠네', 0.9079214334487915),
 ('줜나', 0.9029603600502014),
 ('늠', 0.9028109908103943),
 ('너모', 0.8995006680488586),
 ('보기만해도', 0.8941039443016052),
 ('너무..', 0.8938891887664795),
 ('너무너무너무', 0.8874372839927673),
 ('너어무', 0.8868408799171448)]

In [14]:
model_ft.wv.most_similar(['너뮤'])

[('넘넘', 0.923252522945404),
 ('너모', 0.9174432754516602),
 ('너무싸서', 0.9153861403465271),
 ('너무예뻐서', 0.9131459593772888),
 ('너무달아', 0.9068230986595154),
 ('너무귀엽고', 0.9049050211906433),
 ('늠', 0.9035418629646301),
 ('너무덥고', 0.9010043740272522),
 ('너무너무너무', 0.9009068012237549),
 ('너무너무너무너무', 0.9002985954284668)]

## 5. FastText과  Word2Vec 차이 설명하기

3번의 할머니 유사도 비교에서 model_wv는 할머니가 들어간 다른 단어를 출력하지 않지만, model_ft는 '할머니나', '할머니가'처럼  문장성분이 다른 경우를 모두 다른 단어로 여긴다. 4번의 아주나 너무 등의 사례에서도 Word2Vec은 완전완전완전 등의 해당 단어가 반복되는 경우를 제외하지만 fastText는 유사한 단어로 출력한다.
이는 Word2Vec는 단어를 쪼개질 수 없는 단위로 생각하는 반면,  FastText는 하나의 단어를 n-gram으로 생각하여 내부적으로 쪼갤 수 있는 것으로 간주한다. 즉, 내부 단어, 서브워드(subword)를 고려하여 학습한다.
영어의 경우에는 공백을 기준으로 한 단어가 하나의 품사에 대응되지만 한국어에는 공백을 기준으로 단어를 나누었을 때 여러 품사가 섞일 수 있기에 한국어에서는 Word2Vec을 활용하는 것이 유사성을 판단하기에 좋을 것 같다.